<a href="https://colab.research.google.com/github/bryan-wolff/EA619R_2021S1/blob/main/Experimento%202/EA619_Experimento_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EA619 - Laboratório de Análise Linear**
## Experimento 2 - Rádio AM
### Bryan Wolff - RA: 214095
### João Luís Carvalho de Abreu - RA: 175997



## **Importando Bibliotecas**

In [ ]:
!pip install pysoundfile

In [ ]:
#Geral
import math
import numpy as np
import matplotlib.pyplot as plt 
from scipy import signal
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

#Audio
from IPython.display import Audio
from scipy.io import wavfile
import cffi
import librosa
import IPython.display as ipd

#Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#mostrar todas saídas
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
# Configurando o Plotly

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

## **Arquivo de Áudio Original**
Este item restringe-se apenas ao carregamento do arquivo de audio a ser analisado ao longo deste exercício.
Neste caso, utilizou-se o ***librosa***.

In [ ]:
#Leitura de arquivo de áudio do drive

altas = drive.CreateFile({'id':'1KqqUlUatAmAIDvRvVvFflJodvATxGh8F'})
altas.GetContentFile('altas.wav')
baixas = drive.CreateFile({'id':'1SAGEPgEDgyM6wxqcBDeXi_TcxAwaVnkS'})
baixas.GetContentFile('baixas.wav')

fs, station1 = wavfile.read('baixas.wav')
fs, station2 = wavfile.read('altas.wav')

#Vamos garantir que os dois sinais possuem o mesmo número de amostras
station1 = station1[:100000]
station2 = station2[:100000]

#Escutar áudios
print(f'Frequência de Amostragem: {fs}Hz\n')
print("Station1 - Baixas Frequências\n")
ipd.Audio(station1[:100000],rate=fs)
print('\n')
print("Station2 - Altas Frequências\n")
ipd.Audio(station2[:100000],rate=fs)

## **Rotinas**

**Rotinas: Espectro e Plot**

In [ ]:
def espectro(func, fs):
    sinal = np.fft.fft(func) #transf. de Fourier
    sinal = sinal[0:len(sinal)//2] #frequências positivas
    sinal = np.abs(sinal) #módulo do sinal
    w = np.linspace(0,fs/2,sinal.size) #frequencias avaliadas
    return w,sinal

def plot(x, y, title, name_x, name_y):
    #no intuito de obter um gráfico interativo para a visualização da principal frequência de cada grupo, utilizou-se a biblioteca plotly
    configure_plotly_browser_state()
    init_notebook_mode(connected = False)

    fig = go.Figure(data=go.Scatter(x=x, 
                                    y=y,
                                    mode='lines',
                                    line=dict(color='#6752de')),
                    layout=go.Layout(title=title,
                                     xaxis=dict(title=name_x),
                                     yaxis=dict(title=name_y),
                                     plot_bgcolor='#f2f0f0'))
    fig.show()


## Frequência da portadora

Suponhando que a maior frequência presente nos dois sinais seja 3 kHz e que o alto falante reproduza frequências até 12 kHz, a maior frequência do sinal modulado será dada por $f_c + f_m$, sendo $f_c$ a frequência da portadora e $f_m$ a frequência do sinal. 

Neste caso, temos que $f_m= 3KHz$, logo:
$f_c + 3 < 12KHz \Longleftrightarrow f_c < 9HKz$

Nessa perspectiva, a maior frequência da portadora para que o sinal modulado ainda possa ser transmitido é de $9KHz$.

## **Espectros dos Sinais Originais**



A partir das funções implementadas no tópico anterior (Rotinas), será obtido o espectro do sinal apenas com as frequências positivas.

In [ ]:
w_baixa, sinal_baixa = espectro(station1, fs)
plot(w_baixa, sinal_baixa, 'Espectro do Sinal de Baixas Frequências', 'Freq. (Hz)', 'Amplitude')

In [ ]:
w_alta, sinal_alta = espectro(station2, fs)
plot(w_alta, sinal_alta, 'Espectro do Sinal de Altas Frequências', 'Freq. (Hz)', 'Amplitude')

Ao observar os espectros gerados, é notável que em ambos os sinais as principais frequências se encontram abaixo de $3KHz$, sendo possivel então, filtráços a partir de um filtro passa baixas de $3KHz$ sem ocorrer em perdas.

## **Espectros Filtrados**

Nesta secção foi projetado um um filtro passa baixa, mais especificamente Filtro de Butterworth para filtrar os sinais e eliminar ruídos, que atrapalhariam no processo de modulação de transmissão.

**Filtro de Butterworth**

In [ ]:
#Projeta o filtro
sos = signal.butter(8, 3000, 'low', fs = fs, output = 'sos')

#Plota a resposta em frequência do filtro
w,h = signal.sosfreqz(sos,fs = fs)

plt.figure(figsize=(10,6))
plt.grid()
plt.plot(w,np.abs(h), color = '#0ccf08')
plt.xlim((0,4000))
plt.xlabel('Frequência (Hz)')
plt.ylabel('$|H(f)|$')
plt.title('Filtro passa baixas de Butterworth')
plt.show()

#Filtra os sinais usando o filtro projetado acima
station2_filtrado = signal.sosfilt(sos,station2)
station1_filtrado = signal.sosfilt(sos,station1)

Os espectros filtrados pelo Filtro de Butterworth são plotados logo abaixo.

In [ ]:
w_baixa_filtrado, sinal_baixa_filtrado = espectro(station1_filtrado, fs)
plot(w_baixa_filtrado, sinal_baixa_filtrado, 'Espectro do Sinal de Baixas Frequências Filtrado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
w_alta_filtrado, sinal_alta_filtrado = espectro(station2_filtrado, fs)
plot(w_alta_filtrado, sinal_alta_filtrado, 'Espectro do Sinal de Altas Frequências Filtrado', 'Freq. (Hz)', 'Amplitude')

Ao análisar os espectros filtrados pelo Filtro de Butterworth, é notável que foi removido a maioria (o Filtro não é ideal) das componentes acima de 3KHz, nos permitindo transmitir um dos sinais com uma portadora de frequência 8KHz sem que haja interferência.

## **Modulação**

Para modular o sinal station2, o multiplicamos por $cos(2\pi f_ct)$ com $f_c = 8KHz$.

In [ ]:
# número de amostras / fs = 100000/44100 = 2.26
t = np.linspace(0, 100000/fs, 100000)
fc = 8000

portadora = [np.cos(2*np.pi*fc*i) for i in t]
station2_filtrado_cos = [station2_filtrado[i]*portadora[i] for i in range(len(t))]

plot(t, station2_filtrado_cos, 'Sinal de Altas Frequências Modulado', 'Tempo (s)', '')

In [ ]:
w_cos, sinal_cos = espectro(station2_filtrado_cos, fs)
plot(w_cos, sinal_cos, 'Espectro do Sinal de Altas Frequências Modulado', 'Freq. (Hz)', 'Amplitude')

## **Transmissão**

Para transmiti-los, será somado os sinais station1 e station2 modulado

In [ ]:
sinal_transmitido = station1 + station2_filtrado_cos

ipd.Audio(sinal_transmitido, rate = fs)

In [ ]:
w_transm, sinal_transm = espectro(sinal_transmitido, fs)
plot(w_transm, sinal_transm, 'Espectro do Sinal Transmitido', 'Freq. (Hz)', 'Amplitude')

Ao comparar este sinal com o sinal rx dado, é notável que os áudios são semelhantes.

In [ ]:
rx = drive.CreateFile({'id':'1PK5SffdhoB0w09hSsYisjZO8q9thkzDZ'})
rx.GetContentFile('rx.wav')

#rx = tx
fs, rx = wavfile.read('rx.wav')

ipd.Audio(rx[:100000],rate=fs)

In [ ]:
w_rx, sinal_rx = espectro(rx, fs)
plot(w_rx, sinal_rx, 'Espectro do Sinal rx', 'Freq. (Hz)', 'Amplitude')

## **Sinais Recuperados**

### **Rec. Sinal RX**

#### **Sinal 1**

In [ ]:
rec1 = signal.sosfilt(sos,rx)
w_rec1, sinal_rec1 = espectro(rec1, fs)
plot(w_rec1, sinal_rec1, 'Espectro do Sinal 1 Recuperado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
ipd.Audio(rec1[:100000],rate=fs)

#### **Sinal 2**

In [ ]:
#Projetar um filtro passa faixa
passa_faixa = signal.butter(8, (4000, 12000), btype = 'bandpass', fs = fs, output = 'sos')

sinal_rx_isolado = signal.sosfilt(passa_faixa, rx)

In [ ]:
w_iso, sinal_iso = espectro(sinal_rx_isolado, fs)
plot(w_iso, sinal_iso, 'Sinal Transmitido Filtrado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
sinal_isolado_modulado = [sinal_rx_isolado[i]*portadora[i] for i in range(len(t))]

w_mod, sinal_mod = espectro(sinal_isolado_modulado, fs)
plot(w_mod, sinal_mod, 'Espectro do Sinal Modulado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
rec_2 = signal.sosfilt(sos, sinal_isolado_modulado)

w_rec2, sinal_rec2 = espectro(rec_2, fs)
plot(w_rec2, sinal_rec2, 'Espectro do Sinal 2 Recuperado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
ipd.Audio(rec_2[:100000], rate=fs)

### **Rec. Sinal Somado**

#### **Sinal 1**

Para recuperar o sinal 1, utilizaremos os sinais somados passsando-o por um mesmo filtro passa baixas com frequência de corte de 3KHz utilizado anteriormente.

In [ ]:
rec1 = signal.sosfilt(sos,sinal_transmitido)
w_rec1, sinal_rec1 = espectro(rec1, fs)
plot(w_rec1, sinal_rec1, 'Espectro do Sinal 1 Recuperado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
ipd.Audio(rec1[:100000],rate=fs)

#### **Sinal 2**

In [ ]:
#Projetar um filtro passa faixa
passa_faixa = signal.butter(8, (4000, 12000), btype = 'bandpass', fs = fs, output = 'sos')

sinal_transmitido_isolado = signal.sosfilt(passa_faixa, sinal_transmitido)

In [ ]:
w_iso, sinal_iso = espectro(sinal_transmitido_isolado, fs)
plot(w_iso, sinal_iso, 'Sinal Transmitido Filtrado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
sinal_isolado_modulado = [sinal_transmitido_isolado[i]*portadora[i] for i in range(len(t))]

w_mod, sinal_mod = espectro(sinal_isolado_modulado, fs)
plot(w_mod, sinal_mod, 'Espectro do Sinal Modulado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
rec_2 = signal.sosfilt(sos, sinal_isolado_modulado)

w_rec2, sinal_rec2 = espectro(rec_2, fs)
plot(w_rec2, sinal_rec2, 'Espectro do Sinal 2 Recuperado', 'Freq. (Hz)', 'Amplitude')

In [ ]:
ipd.Audio(rec_2[:100000], rate=fs)